In [1]:
# Install BigQuery client library (run this cell once)
%pip install google-cloud-bigquery --quiet
%pip install bigquery-magics --quiet
%pip install db-dtypes --quiet
# Import libraries
from google.cloud import bigquery
import pandas as pd
%load_ext bigquery_magics
import db_dtypes
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "salesforce-465614-2cf9e37da64b.json"
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


<b>Entry in ETL Job Monitoring table</b>

In [ ]:
from datetime import datetime
from google.cloud import bigquery
# Set monitoring table and module name
monitoring_table = "salesforce-465614.cust_analytics.etl_job_monitoring"
module_name = "Stage_to_Persistent_Layer"
job_run_date = datetime.utcnow().date()
start_time = datetime.utcnow()
# Insert start record
insert_start = f"""
INSERT INTO `{monitoring_table}` (job_run_date, module_name, start_time, status)
VALUES (DATE('{job_run_date}'), '{module_name}', TIMESTAMP('{start_time}'), 'Running')
"""
client.query(insert_start).result()

C:\Users\Home\AppData\Local\Temp\ipykernel_23652\2317656597.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  job_run_date = datetime.utcnow().date()
C:\Users\Home\AppData\Local\Temp\ipykernel_23652\2317656597.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = datetime.utcnow()


<b>Stage to Persistent Layer</b>

In [ ]:
from google.cloud import bigquery
#Load to Dim_Customers
dim_customer_query = f"""
MERGE `salesforce-465614.cust_analytics.dim_customers` T
USING `salesforce-465614.cust_analytics.stage_customers` S
ON T.customer_id = S.customer_id AND T.is_current = TRUE
WHEN MATCHED AND (
    T.first_name != S.first_name OR
    T.last_name != S.last_name OR
    T.gender != S.gender OR
    T.age != S.age OR
    T.signup_date != S.signup_date OR
    cast(T.loyalty_program as string) != CAST(S.loyalty_program AS string) OR
    T.email != S.email OR
    T.city != S.city OR
    T.state != S.state OR
    T.country != S.country
)
THEN
  UPDATE SET
    T.end_date = CURRENT_DATE(),
    T.is_current = FALSE

WHEN NOT MATCHED BY TARGET
THEN
  INSERT (
    customer_id, first_name, last_name, gender, age, signup_date, loyalty_program, email, city, state, country, start_date, end_date, is_current
  )
  VALUES (
    S.customer_id, S.first_name, S.last_name, S.gender, S.age, S.signup_date, CAST(S.loyalty_program AS string), S.email, S.city, S.state, S.country, CURRENT_DATE(), NULL, TRUE
  )
"""
# Load to Dim_stores
dim_stores_query = f"""
MERGE `salesforce-465614.cust_analytics.dim_stores` T
USING `salesforce-465614.cust_analytics.stage_stores` S
ON T.store_id = S.string_field_0 AND T.is_current = TRUE
WHEN MATCHED AND (
    T.store_type != S.string_field_1 OR
    T.store_location != S.string_field_2 OR
    T.region != S.string_field_3
)
THEN
  UPDATE SET
    T.end_date = CURRENT_DATE(),
    T.is_current = FALSE
WHEN NOT MATCHED BY TARGET
THEN
  INSERT (
    store_id, store_type, store_location, region, start_date, end_date, is_current
  )
  VALUES (
    S.string_field_0, S.string_field_1, S.string_field_2, S.string_field_3, CURRENT_DATE(), NULL, TRUE
  )
"""
# Load to dim_products
dim_products_query = f"""
MERGE `salesforce-465614.cust_analytics.dim_products` T
USING `salesforce-465614.cust_analytics.stage_products` S
ON T.product_id = S.product_id AND T.is_current = TRUE
WHEN MATCHED AND (
    T.product_name != S.product_name OR
    T.category != S.category OR
    T.sub_category != S.sub_category OR
    T.price != S.price
)
THEN
  UPDATE SET
    T.end_date = CURRENT_DATE(),
    T.is_current = FALSE
WHEN NOT MATCHED BY TARGET
THEN
  INSERT (
    product_id, product_name, category, sub_category, price, start_date, end_date, is_current
  )
  VALUES (
    S.product_id, S.product_name, S.category, S.sub_category, S.price, CURRENT_DATE(), NULL, TRUE
  )
"""
fact_transactions_query = f"""
  MERGE `salesforce-465614.cust_analytics.fact_transactions` T
USING `salesforce-465614.cust_analytics.stage_transactions` S
ON T.transaction_id = S.Transaction_id AND T.is_current = TRUE
WHEN MATCHED AND (
    T.customer_id != S.Customer_ID OR
    T.product_id != S.Product_id OR
    T.Store_id != S.Store_id OR
    T.Purchase_Date != S.Purchase_Date OR
    T.Unit_Price != S.Unit_Price OR
    T.quantity != S.Quantity OR
    T.Total_Price != S.Total_Price OR
    T.Payment_Method != S.Payment_Method OR
    T.Order_Status != S.Order_Status
)
THEN
  UPDATE SET
    T.end_date = CURRENT_DATE(),
    T.is_current = FALSE
WHEN NOT MATCHED BY TARGET
THEN
  INSERT (
    transaction_id, customer_id, product_id, Store_id, Purchase_Date, Unit_Price, quantity, Total_Price, Payment_Method, Order_Status, start_date, end_date, is_current
  )
  VALUES (
    S.Transaction_id, S.Customer_ID, S.Product_id, S.Store_id, S.Purchase_Date, S.Unit_Price, S.Quantity, S.Total_Price, S.Payment_Method, S.Order_Status, CURRENT_DATE(), NULL, TRUE
  );
  """


job = client.query(dim_customer_query)
job.result()
print("load from stage_customers to dim_customers completed.")
client.query(dim_stores_query).result()
print("stage_stores to dim_stores load is completed.")
client.query(dim_products_query).result()
print("stage_products to dim_products load is completed.")
client.query(fact_transactions_query).result()
print("Stage_transactions to fact_transactions load is completed.")

try:
    end_time = datetime.utcnow()
    update_success = f"""
    UPDATE `{monitoring_table}`
    SET end_time = TIMESTAMP('{end_time}'), status = 'Success'
    WHERE job_run_date = DATE('{job_run_date}') AND module_name = '{module_name}' AND start_time = TIMESTAMP('{start_time}')
    """
    client.query(update_success).result()
except Exception as e:
    # Update status as Failed
    fail_time = datetime.utcnow()
    update_failed = f"""
    UPDATE `{monitoring_table}`
    SET end_time = TIMESTAMP('{fail_time}'), status = 'Failed'
    WHERE job_run_date = DATE('{job_run_date}') AND module_name = '{module_name}' AND start_time = TIMESTAMP('{start_time}')
    """
    client.query(update_failed).result()
    print("ETL job failed:", e)
    raise

load from stage_customers to dim_customers completed.
stage_stores to dim_stores load is completed.
stage_products to dim_products load is completed.


C:\Users\Home\AppData\Local\Temp\ipykernel_23652\2727902827.py:89: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow()
